In [8]:
import pandas as pd
from sklearn_pandas import DataFrameMapper
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import imputation, OneHotEncoder, StandardScaler, LabelBinarizer, LabelEncoder, Imputer

In [9]:
df = pd.read_csv("movie_metadata.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
color                        5024 non-null object
director_name                4939 non-null object
num_critic_for_reviews       4993 non-null float64
duration                     5028 non-null float64
director_facebook_likes      4939 non-null float64
actor_3_facebook_likes       5020 non-null float64
actor_2_name                 5030 non-null object
actor_1_facebook_likes       5036 non-null float64
gross                        4159 non-null float64
genres                       5043 non-null object
actor_1_name                 5036 non-null object
movie_title                  5043 non-null object
num_voted_users              5043 non-null int64
cast_total_facebook_likes    5043 non-null int64
actor_3_name                 5020 non-null object
facenumber_in_poster         5030 non-null float64
plot_keywords                4890 non-null object
movie_imdb_link              5043 non-

In [9]:
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [12]:
df.isnull().any()

color                         True
director_name                 True
num_critic_for_reviews        True
duration                      True
director_facebook_likes       True
actor_3_facebook_likes        True
actor_2_name                  True
actor_1_facebook_likes        True
gross                         True
genres                       False
actor_1_name                  True
movie_title                  False
num_voted_users              False
cast_total_facebook_likes    False
actor_3_name                  True
facenumber_in_poster          True
plot_keywords                 True
movie_imdb_link              False
num_user_for_reviews          True
language                      True
country                       True
content_rating                True
budget                        True
title_year                    True
actor_2_facebook_likes        True
imdb_score                   False
aspect_ratio                  True
movie_facebook_likes         False
dtype: bool

In [27]:
df_genres = df.genres.apply(lambda x: x.split('|'))

In [28]:
genres = df_genres.apply(pd.Series)

In [29]:
df_genres.to_frame().head()

,genres
0,"[Action, Adventure, Fantasy, Sci-Fi]"
1,"[Action, Adventure, Fantasy]"
2,"[Action, Adventure, Thriller]"
3,"[Action, Thriller]"
4,[Documentary]


In [40]:
df_genres.index
#df_genres.iterrows()
#df["genres"]
df.iterrows()

<generator object DataFrame.iterrows at 0x7f6d2735c7d8>

In [106]:
def explode(df, df_col):
    rows = []
    for i,row in df.iterrows():
        for element in row[df_col]:
            rows.append([i,element])
    return pd.DataFrame(data=rows, columns=["old_index", df_col])

In [119]:
df_exploded = explode(df_genres.to_frame(), "genres")
#df_exploded.pivot(index=)
df_exploded.head()
df_exploded["val"] = int(1)
df_exploded.head()
df_genres_pivot = df_exploded.pivot(index="old_index", columns="genres")["val"].fillna(int(0))

df_genres_pivot.head()

genres,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
old_index,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
df_genres_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5043 entries, 0 to 5042
Data columns (total 26 columns):
Action         5043 non-null float64
Adventure      5043 non-null float64
Animation      5043 non-null float64
Biography      5043 non-null float64
Comedy         5043 non-null float64
Crime          5043 non-null float64
Documentary    5043 non-null float64
Drama          5043 non-null float64
Family         5043 non-null float64
Fantasy        5043 non-null float64
Film-Noir      5043 non-null float64
Game-Show      5043 non-null float64
History        5043 non-null float64
Horror         5043 non-null float64
Music          5043 non-null float64
Musical        5043 non-null float64
Mystery        5043 non-null float64
News           5043 non-null float64
Reality-TV     5043 non-null float64
Romance        5043 non-null float64
Sci-Fi         5043 non-null float64
Short          5043 non-null float64
Sport          5043 non-null float64
Thriller       5043 non-null float64
War  

In [131]:
df_mod = df.join(df_genres_pivot)
df_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 54 columns):
color                        5024 non-null object
director_name                4939 non-null object
num_critic_for_reviews       4993 non-null float64
duration                     5028 non-null float64
director_facebook_likes      4939 non-null float64
actor_3_facebook_likes       5020 non-null float64
actor_2_name                 5030 non-null object
actor_1_facebook_likes       5036 non-null float64
gross                        4159 non-null float64
genres                       5043 non-null object
actor_1_name                 5036 non-null object
movie_title                  5043 non-null object
num_voted_users              5043 non-null int64
cast_total_facebook_likes    5043 non-null int64
actor_3_name                 5020 non-null object
facenumber_in_poster         5030 non-null float64
plot_keywords                4890 non-null object
movie_imdb_link              5043 non-

In [235]:
imputerBla = Imputer(copy=False)

mapper = DataFrameMapper([
 (['duration', 'budget'], [Imputer(), StandardScaler()],
) 
],df_out=False)



#mapper = DataFrameMapper([
 #   ('color', LabelBinarizer()),
 #   ('director_name', LabelBinarizer()),
 #   ('budget_imputed', StandardScaler()),
 #   ('duration_imputed', StandardScaler())
#])

pipelines = Pipeline([
    ('imputer', imputers),
    ('mapper', mapper)
])

In [237]:
mapper.fit_transform(df)

#>>> mapper3 = DataFrameMapper([
#...     (['ae'], [sklearn.preprocessing.Imputer(),
#...                sklearn.preprocessing.StandardScaler()])])
#>>> data_3 = pd.DataFrame({'age': [1, np.nan, 3]})
#>>> mapper3.fit_transform(data_3)
#array([[-1.22474487],
#       [ 0.        ],
#       [ 1.22474487]])

array([[ 2.81423447,  1.00748977],
       [ 2.45648737,  1.32927784],
       [ 1.62174415,  1.04835174],
       ..., 
       [-1.24023262, -0.20303919],
       [-0.28624037,  0.        ],
       [-0.68373714, -0.20304072]])

In [221]:
df_imputed = imputers.transform(df)
df_imputed

/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


,children_imputed_0,children_imputed_1,children_imputed_2,children_imputed_3,children_imputed_4,children_imputed_5,children_imputed_6,children_imputed_7,children_imputed_8,children_imputed_9,...,budget_imputed_4541,budget_imputed_4542,budget_imputed_4543,budget_imputed_4544,budget_imputed_4545,budget_imputed_4546,budget_imputed_4547,budget_imputed_4548,budget_imputed_4549,budget_imputed_4550
0,178.0,169.0,148.0,164.0,132.0,156.0,100.0,141.0,153.0,183.0,...,10000.0,1000000.0,200000.0,7000.0,7000.0,7000.0,3250.0,9000.0,1400.0,1100.0


In [ ]:
df.head()
mapper.fit(df)

In [188]:
df["duration"].head()

0    178.000000
1    169.000000
2    148.000000
3    164.000000
4    107.201074
Name: duration, dtype: float64

In [193]:
type(df_imputed)

numpy.ndarray

In [196]:
test = StandardScaler()
test.fit(X=df_imputed.T)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin

class SampleExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, vars):
        self.vars = vars  # e.g. pass in a column name to extract

    def transform(self, X, y=None):
        return do_something_to(X, self.vars)  # where the actual feature extraction happens

    def fit(self, X, y=None):
        return self  # generally does nothing

# Custom Transformer for Pipelines

In [114]:
from sklearn.base import TransformerMixin

class ArrayExplodePivot(TransformerMixin):

    def __init__(self, col_name):
        self.col_name = col_name

    def transform(self, X):
        df = self.__explode__(X)
        df_pivot = self.__pivot__(df)
        df_pivot.index.names = ["index"]
        return self.__join__(X, df_pivot)
    
    def fit(self, X):
        return self  # generally does nothing
    
    def __explode__(self, X):
        rows = []
        for i,row in X.iterrows():
            for element in row[self.col_name]:
                rows.append([i,element])
        return pd.DataFrame(data=rows, columns=["old_index", self.col_name])
    
    def __pivot__(self, X):
        X["val"] = int(1)
        return X.pivot(index="old_index", columns=self.col_name)["val"].fillna(int(0))
    
    def __join__(self, X, Y):
        return X.join(Y)

    
class StringToArray(TransformerMixin):
    ''' Takes a Pandas Dataframe and two column names as input and transforms the input column '''
    def __init__(self, col_name_in, col_name_out, seperator):
        self.col_name_in = col_name_in  # e.g. pass in a column name to extract
        self.col_name_out = col_name_out  # e.g. pass in a column name to extract
        self.seperator = seperator

    def transform(self, X):
        X[self.col_name_out] = X[self.col_name_in].apply(lambda x: x.split('|'))
        return X # where the actual feature extraction happens
    
    def __join__(self, X, Y):
        return X.join(Y)

    def fit(self, X):
        return self  # generally does nothing

In [115]:
pipeTest = Pipeline([
    ('split', StringToArray("genres", "genres_out", "|")),
    ('explode', ArrayExplodePivot("genres_out"))
])

In [116]:
df_test = df
pipeTest.transform(df_test).head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
# extract the step 2
pipeTest.steps[0][1].__doc__

' Takes a Pandas Dataframe and two column names as input and transforms the input column '